In [16]:
## Data Injection
from langchain_community.document_loaders import TextLoader
loader= TextLoader("speech.txt")
text_documents= loader.load()
text_documents

[Document(metadata={'source': 'speech.txt'}, page_content='The Power of Persistence\n\nMy friends, today I want to talk to you about one of the most important qualities for success: persistence.\n\nLife will knock you down - this is inevitable. But what separates those who achieve their dreams from those who don\'t is the willingness to get back up. Every failure is a lesson, every setback is setup for a comeback.\n\nConsider Thomas Edison\'s story. He failed thousands of times before inventing the light bulb. When asked about these failures, he famously replied: "I have not failed. I\'ve just found 10,000 ways that won\'t work." That is the mindset of persistence.\n\nIn your own journey:\n1. When you face obstacles, see them as opportunities to grow\n2. When you feel discouraged, remember your why\n3. When progress seems slow, trust the process\n\nPersistence isn\'t about never falling - it\'s about always rising. It\'s not about speed, but direction. Keep moving forward, one step at 

In [17]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")


In [18]:
# Web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4

#load, chunk, and index the content of the html page
loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                      bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                          class_=("post-title","post_content","post_header")

                      )))

text_documents= loader.load()

In [19]:
##Pdf reader 
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs= loader.load()

In [20]:
## Transform
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents= text_splitter.split_documents(docs)
documents[:5]


[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu'),
 Document(metadata={'producer': 'pd

In [21]:
from langchain_ollama import OllamaEmbeddings
from langchain_groq import ChatGroq
from langchain_community.vectorstores import Chroma

# Initialize Ollama Embeddings (make sure Ollama is running locally)
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",  # or another embedding model like "nomic-embed-text"
)


# Initialize ChatGroq LLM (for chat/completion)
llm = ChatGroq(
    model="llama3-8b-8192",  # Updated model name
    temperature=0.7
)

In [22]:
# # Create vector store with the embeddings
# db = Chroma.from_documents(
#     documents=documents[:15],  # Your loaded documents
#     embedding=embeddings,     # Use the embedding model here
#     persist_directory="./chroma_db"  # Optional: persist the database
# )


# # Process documents in smaller batches
# batch_size = 5  # Reduce if still crashing
# for i in range(0, len(documents), batch_size):
#     _ = Chroma.from_documents(
#         documents=documents[i:i+batch_size],
#         embedding=embeddings,
#         persist_directory="./chroma_db"
#     )

In [23]:
from langchain_community.vectorstores import FAISS  # More memory-efficient

db = FAISS.from_documents(
    documents=documents[:15],
    embedding=embeddings
)
db.save_local("./faiss_db")  # Save instead of persist_directory

In [24]:
# Example usage:
query = "What is the main theme of the speech?"
docs = db.similarity_search(query)

# Use the LLM to generate a response
response = llm.invoke(f"Based on these documents: {docs}, answer: {query}")
print(response.content)

Based on the provided documents, it appears that the main theme of the speech is "Attention" in the context of natural language processing and deep learning. The documents seem to be a research paper or article discussing various aspects of attention mechanisms in neural networks, including self-attention, multi-head attention, and their applications in tasks such as reading comprehension, abstractive summarization, and language modeling.


In [25]:
## Vector database
query= "Who is the authors of the attention is all you need reseach paper"
result= db.similarity_search(query)
result

[Document(id='3f84ce69-4192-4036-975a-ac666b4fb37e', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'keywords': '', 'moddate': '2024-04-10T21:11:43+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto

In [26]:
## Vector database
query= "What is attention is all you need"
result= db.similarity_search(query)
result[0].page_content

'reduced to a constant number of operations, albeit at the cost of reduced effective resolution due\nto averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as\ndescribed in section 3.2.\nSelf-attention, sometimes called intra-attention is an attention mechanism relating different positions\nof a single sequence in order to compute a representation of the sequence. Self-attention has been'

In [27]:
# ## FAISS Vector Database
# from langchain_community.vectorstores import FAISS
# db1= FAISS.from_documents(documents[:20], embedding=llm)